In [5]:
import pandas as pd
import numpy as np
import math

import plotly.graph_objects as go
import plotly.express as px

import dash
import dash_core_components as dcc
import dash_html_components as html
#import dash_daq as daq
from dash.dependencies import Input, Output
import requests
import json


# create dashborad
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets, serve_locally=False)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}
SCORE_API_URL ='http://127.0.0.1:5000/api/score/'
# Boostrap CSS.
app.css.append_css({'external_url': 'https://codepen.io/amyoshino/pen/jzXypZ.css'})

## load dataFrames
#df_scores = pd.read_pickle("../df_scores.pkl")
df_feat_imp = pd.read_pickle("../df_feat_imp.pkl")
df_test = pd.read_pickle("../df_test.pkl")
# send request to score API in order to get scores dataFrame
response = requests.get(SCORE_API_URL)
jsonified_df_scores = response.content.decode('utf-8')
df_scores = pd.read_json(jsonified_df_scores, orient='split')
    
fig0 = px.histogram(df_test, x=df_feat_imp.index[0], color=df_scores['class'])

fig0.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

fig1 = px.histogram(df_test, x=df_feat_imp.index[1], color=df_scores['class'])

fig1.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

fig2 = px.histogram(df_test, x=df_feat_imp.index[2], color=df_scores['class'])

fig2.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

fig3 = px.histogram(df_test, x=df_feat_imp.index[3], color=df_scores['class'])

fig3.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

fig4 = px.histogram(df_test, x=df_feat_imp.index[4], color=df_scores['class'])

fig4.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.Div([
        html.Img(
            src="https://www.digitalnolimit.blog/wp-content/uploads/2018/05/openclassrooms-levee-fonds-startup.png",
            className = "one columns",
            style={
                'height': '6%',
                'width': '6%',
                'padding': '10px 10px 10px 10px'
            },
        ),
        html.H1(
            children='Dashboard',
            className = "ten columns",
            style={
                'textAlign': 'center',
                'color': 'red'
            }
        ),

        html.Div(children='Home Credit Default Risk', className = "ten columns", style={
            'textAlign': 'center',
            'color': 'white',
            'font-style': 'italic',
            'padding-bottom': '30px'
        }),

    ], className = "row"),
    # html div including scores and two most importants features histo
    html.Div([
        html.Div([
            html.H6("Enter the ID of loan (SK_ID_CURR):", style={'color': colors['text']}),
            html.Div(["Loan ID: ",
                      dcc.Input(id='input_LoanId', value=df_test.SK_ID_CURR[0], type="number")], style={'color': colors['text']}),
            html.Br(),
            html.Div(id='customer_scores', style={'color': colors['text']}),
        ], className = "six columns", style={'text-align': 'center', 'padding': '150px','border-style': 'dashed double none'}),
        html.Div([
            dcc.Graph(id='feat_imp_0', figure=fig0),
        ], className = "six columns", style={'border-style': 'dashed double none'}),
           
    ], className = "row"),
    
    html.Div([
        html.Div([
            dcc.Graph(id='feat_imp_1', figure=fig1),
        ], className = "six columns", style={'border-style': 'dashed double none'}),
        html.Div([
            dcc.Graph(id='feat_imp_2', figure=fig2),
        ], className = "six columns", style={'border-style': 'dashed double none'}),
    ], className = "row"),
    
    html.Div([
        html.Div([
            dcc.Graph(id='feat_imp_3', figure=fig3),
        ], className = "six columns", style={'border-style': 'dashed double none'}),
        html.Div([
            dcc.Graph(id='feat_imp_4', figure=fig4),
        ], className = "six columns", style={'border-style': 'dashed double none'}),
    ], className = "row"),

])

@app.callback(
    Output(component_id='customer_scores', component_property='children'),
    Input(component_id='input_LoanId', component_property='value')
)
def update_scores(input_value):
    loan_idx = df_test.SK_ID_CURR[df_test.SK_ID_CURR == input_value].index
    score = round(df_scores.iloc[loan_idx,0].values[0],3)
    label = df_scores.iloc[loan_idx,1].values[0]
    if label == 1:
        loanStatus = 'Refused'
    else:
        loanStatus = 'Accepted'
    
    return 'customer score: {} {}  Loan request: {}'.format(score,">>", loanStatus)

@app.callback(
    Output(component_id='feat_imp_0', component_property='figure'),
    Output(component_id='feat_imp_1', component_property='figure'),
    Output(component_id='feat_imp_2', component_property='figure'),
    Output(component_id='feat_imp_3', component_property='figure'),
    Output(component_id='feat_imp_4', component_property='figure'),
    Input(component_id='input_LoanId', component_property='value')
)
def update_fig(input_value):
    x0 = df_test[df_test.SK_ID_CURR == input_value][df_feat_imp.index[0]].values[0]
    x1 = df_test[df_test.SK_ID_CURR == input_value][df_feat_imp.index[1]].values[0]
    x2 = df_test[df_test.SK_ID_CURR == input_value][df_feat_imp.index[2]].values[0]
    x3 = df_test[df_test.SK_ID_CURR == input_value][df_feat_imp.index[3]].values[0]
    x4 = df_test[df_test.SK_ID_CURR == input_value][df_feat_imp.index[4]].values[0]
    fig0.add_vline(x=x0, line_width=3, line_dash="dash", line_color="green")
    fig1.add_vline(x=x1, line_width=3, line_dash="dash", line_color="green")
    fig2.add_vline(x=x2, line_width=3, line_dash="dash", line_color="green")
    fig3.add_vline(x=x3, line_width=3, line_dash="dash", line_color="green")
    fig4.add_vline(x=x4, line_width=3, line_dash="dash", line_color="green")
    return fig0, fig1, fig2, fig3, fig4
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [ ]:
    dcc.Graph(
        id='feat_imp_1',
        figure=fig1
    )
    
    dcc.Graph(
        id='feat_imp_2',
        figure=fig2
    )
    
    dcc.Graph(
        id='feat_imp_3',
        figure=fig3
    )
    
    dcc.Graph(
        id='feat_imp_4',
        figure=fig4
    )